Implementation of 2D elasto-plastic problem using FEniCS-X. The code is based on a [legacy solution](https://comet-fenics.readthedocs.io/en/latest/demo/2D_plasticity/vonMises_plasticity.py.html) for FEniCS 2019.

In [1]:
import meshio
import numpy as np

import ufl
from dolfinx import fem, io, plot
# from ufl import ds, dx, grad, 

from mpi4py import MPI
from petsc4py.PETSc import ScalarType
# import pyvista
from petsc4py import PETSc

In [2]:
def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={"name_to_read":[cell_data]})
    return out_mesh

In [3]:
#It works with the msh4 only!!
msh = meshio.read("thick_cylinder.msh")

# Create and save one file for the mesh, and one file for the facets 
triangle_mesh = create_mesh(msh, "triangle", prune_z=True)
line_mesh = create_mesh(msh, "line", prune_z=True)
meshio.write("thick_cylinder.xdmf", triangle_mesh)
meshio.write("mt.xdmf", line_mesh)

with io.XDMFFile(MPI.COMM_WORLD, "thick_cylinder.xdmf", "r") as xdmf:
    mesh = xdmf.read_mesh(name="Grid")
    ct = xdmf.read_meshtags(mesh, name="Grid")

mesh.topology.create_connectivity(mesh.topology.dim, mesh.topology.dim - 1)

with io.XDMFFile(MPI.COMM_WORLD, "mt.xdmf", "r") as xdmf:
    ft = xdmf.read_meshtags(mesh, name="Grid")

In [4]:
# elastic parameters
E = 70e3
nu = 0.3
lmbda = E*nu/(1+nu)/(1-2*nu)
mu = fem.Constant(mesh, ScalarType(E/2./(1+nu)))
sig0 = 250.  # yield strength
Et = E/100.  # tangent modulus
H = E*Et/(E-Et)  # hardening modulus

Re, Ri = 1.3, 1.   # external/internal radius
ds = ufl.Measure("ds", domain=mesh)

In [5]:
deg_u = 2
deg_stress = 2
V = fem.VectorFunctionSpace(mesh, ("CG", deg_u))
We = ufl.VectorElement("Quadrature", mesh.ufl_cell(), degree=deg_stress, dim=4, quad_scheme='default')
W0e = ufl.FiniteElement("Quadrature", mesh.ufl_cell(), degree=deg_stress, quad_scheme='default')
W = fem.FunctionSpace(mesh, We)
W0 = fem.FunctionSpace(mesh, W0e)

In [6]:
sig = fem.Function(W)
sig_old = fem.Function(W)
n_elas = fem.Function(W)
beta = fem.Function(W0)
p = fem.Function(W0, name="Cumulative plastic strain")
u = fem.Function(V, name="Total displacement")
du = fem.Function(V, name="Iteration correction")
Du = fem.Function(V, name="Current increment")
v = ufl.TrialFunction(V)
u_ = ufl.TestFunction(V)

In [7]:
left_marker = 3
down_marker = 1
left_facets = ft.indices[ft.values == left_marker]
down_facets = ft.indices[ft.values == down_marker]
left_dofs = fem.locate_dofs_topological(V.sub(0), mesh.topology.dim-1, left_facets)
down_dofs = fem.locate_dofs_topological(V.sub(1), mesh.topology.dim-1, down_facets)

bcs = [fem.dirichletbc(ScalarType(0), left_dofs, V.sub(0)), fem.dirichletbc(ScalarType(0), down_dofs, V.sub(1))]

In [8]:
n = ufl.FacetNormal(mesh)
q_lim = float(2/np.sqrt(3)*np.log(Re/Ri)*sig0)

class Loading:
    def __init__(self):
        self.q = q_lim
        self.t = 0.0

    def eval(self, x):
        return np.full(x.shape[1], -self.q * self.t)

V_real = fem.FunctionSpace(mesh, ("CG", 2))
loading = Loading()
load_func = fem.Function(V_real)
load_func.interpolate(loading.eval)

def F_ext(v):
    return load_func * ufl.inner(n, v)*ds(4)

In [9]:
def eps(v):
    e = ufl.sym(ufl.grad(v))
    return ufl.as_tensor([[e[0, 0], e[0, 1], 0],
                          [e[0, 1], e[1, 1], 0],
                          [0, 0, 0]])
def sigma(eps_el):
    return lmbda*ufl.tr(eps_el)*ufl.Identity(3) + 2*mu*eps_el

def as_3D_tensor(X):
    return ufl.as_tensor([[X[0], X[3], 0],
                          [X[3], X[1], 0],
                          [0, 0, X[2]]])

In [10]:
ppos = lambda x: (x+abs(x))/2.
def proj_sig(deps, old_sig, old_p):
    sig_n = as_3D_tensor(old_sig)
    sig_elas = sig_n + sigma(deps)
    s = ufl.dev(sig_elas)
    sig_eq = ufl.sqrt(3/2.*ufl.inner(s, s))
    f_elas = sig_eq - sig0 - H*old_p
    dp = ppos(f_elas)/(3*mu+H)
    n_elas = s/sig_eq*ppos(f_elas)/f_elas
    beta = 3*mu*dp/sig_eq
    new_sig = sig_elas-beta*s
    return ufl.as_vector([new_sig[0, 0], new_sig[1, 1], new_sig[2, 2], new_sig[0, 1]]), \
           ufl.as_vector([n_elas[0, 0], n_elas[1, 1], n_elas[2, 2], n_elas[0, 1]]), \
           beta, dp

In [11]:
def sigma_tang(e):
    N_elas = as_3D_tensor(n_elas)
    # return sigma(e) - 3*mu*(3*mu/(3*mu+H)-beta)*inner(N_elas, e)*N_elas 
    return sigma(e) - 3*mu*(3*mu/(3*mu+H)-beta)*ufl.inner(N_elas, e)*N_elas - 2*mu*beta*ufl.dev(e)

In [12]:
dx = ufl.Measure(
    "dx",
    domain=mesh
    # metadata={"quadrature_degree": deg_stress, "quadrature_scheme": "default"},
)

a_Newton = ufl.inner(eps(v), sigma_tang(eps(u_)))*dx
res = -ufl.inner(eps(u_), as_3D_tensor(sig))*dx 
# + F_ext(u_)

In [13]:
ufl.dev(eps(u_)).ufl_shape

(3, 3)

In [56]:
# def local_project(v, V, u=None):
#     dv = ufl.TrialFunction(V)
#     v_ = ufl.TestFunction(V)
#     a_proj = inner(dv, v_)*dxm
#     b_proj = inner(v, v_)*dxm
#     solver = LocalSolver(a_proj, b_proj)
#     solver.factorize()
#     if u is None:
#         u = fem.Function(V)
#         solver.solve_local_rhs(u)
#         return u
#     else:
#         solver.solve_local_rhs(u)
#         return

In [14]:
P0 = fem.FunctionSpace(mesh, ("DG", 0))
p_avg = fem.Function(P0, name="Plastic strain")

In [15]:
A = fem.petsc.assemble_matrix(fem.form(a_Newton), bcs=bcs)
# Res = fem.petsc.create_vector(fem.form(res)
# solver = PETSc.KSP().create(mesh.comm)
# solver.setOperators(A)
# solver.setType(PETSc.KSP.Type.PREONLY)
# solver.getPC().setType(PETSc.PC.Type.LU)

ValueError: Mismatch of tabulation points and element points.

In [ ]:
Nitermax, tol = 200, 1e-8  # parameters of the Newton-Raphson procedure
Nincr = 20
load_steps = np.linspace(0, 1.1, Nincr+1)[1:]**0.5
results = np.zeros((Nincr+1, 2))
for (i, t) in enumerate(load_steps):
    loading.t = t
    # A, Res = assemble_system(a_Newton, res, bcs)
    A = fem.petsc.assemble_matrix(a_Newton, bcs=bcs)
    A.assemble()
    Res = fem.petsc.create_vector(res)
    # uh = fem.Function(V)
    nRes0 = Res.norm("l2")
    nRes = nRes0
    Du.interpolate(fem.Constant(mesh, (ScalarType(0), ScalarType(0))))
    print("Increment:", str(i+1))
    niter = 0
    while nRes/nRes0 > tol and niter < Nitermax:
        solve(A, du.vector(), Res, "mumps")
        Du.assign(Du+du)
        deps = eps(Du)
        sig_, n_elas_, beta_, dp_ = proj_sig(deps, sig_old, p)
        local_project(sig_, W, sig)
        local_project(n_elas_, W, n_elas)
        local_project(beta_, W0, beta)
        A, Res = assemble_system(a_Newton, res, bcs)
        nRes = Res.norm("l2")
        print("    Residual:", nRes)
        niter += 1
    u.assign(u+Du)
    sig_old.assign(sig)
    p.assign(p+local_project(dp_, W0))